# Segmenting and Clustering Neighborhoods in Toronto
This notebook is used for the pear-graded assignment of the data science capstone course, week 3.

there are tree sub assignments to do. they will be marked by titles for quick reference:
    - assignment 1
    - assignment 2
    - assignment 3

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [ ]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: \ 
  - anaconda::ca-certificates-2017.08.26-h1d4fec5_0, anaconda::openssl-1.0.2n-hb7f436b_0
  - anaconda::ca-certificates-2017.08.26-h1d4fec5_0, defaults::openssl-1.0.2n-hb7f436b_0
  - anaconda::openssl-1.0.2n-hb7f436b_0, defaults::ca-certificates-2017.08.26-h1d4fec5_0
  - defaults::ca-certificates-2017.08.26-h1d4fec5_0, defaults::openssl-1.0.2n-hb7f436bdone


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: / 

## assignment 1
In this assignment we wil scrape date from the following wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and insert it in an pandas dataframe.

In [ ]:
#first we need to install the BeautifulSoup package
!conda install -c anaconda beautifulsoup4 --yes

To get the dataframe into pandas we will use beautifulsoup to scrape the website and turn it into a dataframe. 

In [2]:
from bs4 import BeautifulSoup
# Here, we're just importing Beautiful Soup
page_link = 'https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# this is the url.
page_response = requests.get(page_link, timeout=5)
# here, we fetch the content from the url, using the requests library
soup = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.
table = soup.find_all('table')
df = pd.read_html(str(table), header=0)[0]

Now we got the raw table from the website into a pandas dataframe. the next step is to explore and clean the data.

In [3]:
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
#first we delete the rows where there is no Borough
# Get names of indexes for which column Borough is 'Not assigned'
indexNames = df[ df['Borough'] == 'Not assigned' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In the database produced the Neighborhoods are already combined if they have the same Postal code. they are separated bij '/' instead of ',' so this will need to be replaced.

In [5]:
#Replace the ' /' by ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
print('Are there any duplicate values in Postal code:', df['Postal code'].duplicated().any())
df.describe()

Are there any duplicate values in Postal code: False


,Postal code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M4C,North York,Downsview
freq,1,24,4


As shown below there are no cells left in the dataframe with a NaN value.

In [6]:
df.isnull().sum(axis = 0)

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [7]:
df.shape

(103, 3)

## Assignment 2
In assignment 2 we are going to add the latlong coordinates of the postal codes.

unfortunatly the geocoder package is not working so the CSV will be used.

In [31]:
#first import the CSV file
Postal_code = pd.read_csv('https://cocl.us/Geospatial_data')
Postal_code.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#Then we join the two files and check if there are no empty values, the amount of rows are still the same and show the head of the dataframe
df_ll = df.join(Postal_code.set_index('Postal Code'), on='Postal code')
print(df_ll.isnull().sum(axis = 0))
print(df_ll.shape)
df_ll.head(10)

Postal code     0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64
(103, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
